<a href="https://colab.research.google.com/github/Darshil-N/NeroBot/blob/main/Copy_of_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers accelerate bitsandbytes peft datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.0 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import warnings
warnings.filterwarnings('ignore')

# Configure 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [3]:
model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"  # Using DeepSeek Coder as it's better for SQL
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [4]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("✓ Model loaded successfully!")
print(f"✓ Model: {model_name}")
print(f"✓ Device: {next(model.parameters()).device}")
print(f"✓ Model size: ~{sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")


✓ Model loaded successfully!
✓ Model: deepseek-ai/deepseek-coder-6.7b-instruct
✓ Device: cuda:0
✓ Model size: ~3.50B parameters


In [8]:
print("=" * 60)
print("STEP 2: Creating Training Dataset")
print("=" * 60)

from datasets import Dataset
import pandas as pd

# Database schema definition
schema_definition = """
CREATE TABLE cameras_table (
    camera_id SERIAL PRIMARY KEY,
    location TEXT,
    coverage_area TEXT,
    status TEXT CHECK (status IN ('Active', 'Inactive'))
);

CREATE TABLE events_table (
    event_id SERIAL PRIMARY KEY,
    timestamp TIMESTAMPTZ DEFAULT NOW(),
    camera_id INT REFERENCES cameras_table(camera_id),
    location TEXT,
    event_type TEXT CHECK (event_type IN ('fall', 'fight', 'entry', 'crowd', 'violation')),
    anomaly_score FLOAT CHECK (anomaly_score BETWEEN 0 AND 1),
    description TEXT,
    processed BOOLEAN DEFAULT FALSE
);

CREATE TABLE detected_objects_table (
    object_id SERIAL PRIMARY KEY,
    event_id INT REFERENCES events_table(event_id) ON DELETE CASCADE,
    object_type TEXT CHECK (object_type IN ('person', 'vehicle', 'object')),
    confidence_score FLOAT CHECK (confidence_score BETWEEN 0 AND 1),
    bounding_box JSONB
);

CREATE TABLE images_table (
    image_id SERIAL PRIMARY KEY,
    event_id INT REFERENCES events_table(event_id) ON DELETE CASCADE,
    file_path TEXT,
    thumbnail_path TEXT,
    captured_at TIMESTAMPTZ DEFAULT NOW()
);
"""

STEP 2: Creating Training Dataset


In [10]:
# Comprehensive training examples
training_data = [
    # Basic SELECT queries
    {
        "question": "Show me all events",
        "sql": "SELECT * FROM events_table;"
    },
    {
        "question": "Get all active cameras",
        "sql": "SELECT * FROM cameras_table WHERE status = 'Active';"
    },
    {
        "question": "List all camera locations",
        "sql": "SELECT camera_id, location FROM cameras_table;"
    },

    # Event type queries
    {
        "question": "Show all fall events",
        "sql": "SELECT * FROM events_table WHERE event_type = 'fall';"
    },
    {
        "question": "Find all fight incidents",
        "sql": "SELECT * FROM events_table WHERE event_type = 'fight';"
    },
    {
        "question": "Get crowd detection events",
        "sql": "SELECT * FROM events_table WHERE event_type = 'crowd';"
    },
    {
        "question": "Show violation events",
        "sql": "SELECT * FROM events_table WHERE event_type = 'violation';"
    },
    {
        "question": "List all entry events",
        "sql": "SELECT * FROM events_table WHERE event_type = 'entry';"
    },

    # Anomaly score queries
    {
        "question": "Show events with high anomaly score above 0.8",
        "sql": "SELECT * FROM events_table WHERE anomaly_score > 0.8;"
    },
    {
        "question": "Find events with anomaly score greater than 0.9",
        "sql": "SELECT * FROM events_table WHERE anomaly_score > 0.9;"
    },
    {
        "question": "Get low confidence events below 0.5",
        "sql": "SELECT * FROM events_table WHERE anomaly_score < 0.5;"
    },

    # Time-based queries
    {
        "question": "Show events from today",
        "sql": "SELECT * FROM events_table WHERE DATE(timestamp) = CURRENT_DATE;"
    },
    {
        "question": "Get events from last 24 hours",
        "sql": "SELECT * FROM events_table WHERE timestamp >= NOW() - INTERVAL '24 hours';"
    },
    {
        "question": "Find events in the last week",
        "sql": "SELECT * FROM events_table WHERE timestamp >= NOW() - INTERVAL '7 days';"
    },
    {
        "question": "Show events between 12:00 and 13:00 today",
        "sql": "SELECT * FROM events_table WHERE DATE(timestamp) = CURRENT_DATE AND EXTRACT(HOUR FROM timestamp) BETWEEN 12 AND 13;"
    },

    # Location-based queries
    {
        "question": "Show events at main entrance",
        "sql": "SELECT * FROM events_table WHERE location = 'main entrance';"
    },
    {
        "question": "Find events in parking lot",
        "sql": "SELECT * FROM events_table WHERE location = 'parking lot';"
    },
    {
        "question": "Get all cameras in building A",
        "sql": "SELECT * FROM cameras_table WHERE location LIKE '%building A%';"
    },

    # JOIN queries
    {
        "question": "Show events with camera details",
        "sql": "SELECT e.*, c.location, c.coverage_area FROM events_table e JOIN cameras_table c ON e.camera_id = c.camera_id;"
    },
    {
        "question": "Get events with detected objects",
        "sql": "SELECT e.event_id, e.event_type, d.object_type, d.confidence_score FROM events_table e JOIN detected_objects_table d ON e.event_id = d.event_id;"
    },
    {
        "question": "List events with images",
        "sql": "SELECT e.event_id, e.event_type, i.file_path FROM events_table e JOIN images_table i ON e.event_id = i.event_id;"
    },

    # Aggregation queries
    {
        "question": "Count total events",
        "sql": "SELECT COUNT(*) FROM events_table;"
    },
    {
        "question": "Count events by type",
        "sql": "SELECT event_type, COUNT(*) as count FROM events_table GROUP BY event_type;"
    },
    {
        "question": "Get average anomaly score",
        "sql": "SELECT AVG(anomaly_score) as avg_score FROM events_table;"
    },
    {
        "question": "Count events per camera",
        "sql": "SELECT camera_id, COUNT(*) as event_count FROM events_table GROUP BY camera_id;"
    },
    {
        "question": "Show events count by location",
        "sql": "SELECT location, COUNT(*) as count FROM events_table GROUP BY location;"
    },

    # Complex queries
    {
        "question": "Show unprocessed high severity events",
        "sql": "SELECT * FROM events_table WHERE processed = FALSE AND anomaly_score > 0.8;"
    },
    {
        "question": "Find fight or fall events with high anomaly score",
        "sql": "SELECT * FROM events_table WHERE event_type IN ('fight', 'fall') AND anomaly_score > 0.8;"
    },
    {
        "question": "Get person detections with high confidence",
        "sql": "SELECT * FROM detected_objects_table WHERE object_type = 'person' AND confidence_score > 0.85;"
    },
    {
        "question": "Show recent unprocessed events",
        "sql": "SELECT * FROM events_table WHERE processed = FALSE AND timestamp >= NOW() - INTERVAL '1 hour';"
    },

    # Description-based queries
    {
        "question": "Find events where students are fighting",
        "sql": "SELECT * FROM events_table WHERE description LIKE '%student%' AND event_type = 'fight';"
    },
    {
        "question": "Show events with assault in description",
        "sql": "SELECT * FROM events_table WHERE description LIKE '%assault%';"
    },
    {
        "question": "Get health issue events",
        "sql": "SELECT * FROM events_table WHERE description LIKE '%health%' OR description LIKE '%dropped%';"
    },

    # ORDER BY queries
    {
        "question": "Show latest events first",
        "sql": "SELECT * FROM events_table ORDER BY timestamp DESC;"
    },
    {
        "question": "List events by highest anomaly score",
        "sql": "SELECT * FROM events_table ORDER BY anomaly_score DESC;"
    },
    {
        "question": "Show oldest unprocessed events",
        "sql": "SELECT * FROM events_table WHERE processed = FALSE ORDER BY timestamp ASC;"
    },

    # LIMIT queries
    {
        "question": "Show last 10 events",
        "sql": "SELECT * FROM events_table ORDER BY timestamp DESC LIMIT 10;"
    },
    {
        "question": "Get top 5 high severity events",
        "sql": "SELECT * FROM events_table ORDER BY anomaly_score DESC LIMIT 5;"
    },

    # UPDATE queries
    {
        "question": "Mark event as processed",
        "sql": "UPDATE events_table SET processed = TRUE WHERE event_id = 1;"
    },
    {
        "question": "Set camera to inactive",
        "sql": "UPDATE cameras_table SET status = 'Inactive' WHERE camera_id = 1;"
    },

    # INSERT queries (based on sample data)
    {
        "question": "Insert a new assault event",
        "sql": "INSERT INTO events_table (camera_id, location, event_type, anomaly_score, description) VALUES (1, 'main entrance', 'fight', 0.89, 'A student1 beating up another student2');"
    },
    {
        "question": "Add a health issue event",
        "sql": "INSERT INTO events_table (camera_id, location, event_type, anomaly_score, description) VALUES (1, 'classroom', 'fall', 0.92, 'Student has dropped');"
    },

    # DELETE queries
    {
        "question": "Delete old processed events",
        "sql": "DELETE FROM events_table WHERE processed = TRUE AND timestamp < NOW() - INTERVAL '30 days';"
    },

    # Multi-condition queries
    {
        "question": "Show fight events from today with high score",
        "sql": "SELECT * FROM events_table WHERE event_type = 'fight' AND DATE(timestamp) = CURRENT_DATE AND anomaly_score > 0.7;"
    },
    {
        "question": "Find active cameras in main building",
        "sql": "SELECT * FROM cameras_table WHERE status = 'Active' AND location LIKE '%main building%';"
    },
]

In [11]:
def format_prompt(question, sql, schema):
    prompt = f"""### Instruction:
You are an SQL expert. Given the following database schema and a natural language question, generate the correct SQL query.

### Database Schema:
{schema}

### Question:
{question}

### SQL Query:
{sql}"""
    return prompt

formatted_data = []
for item in training_data:
    formatted_data.append({
        "text": format_prompt(item["question"], item["sql"], schema_definition)
    })


In [12]:
dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))

print("Dataset structure:")
print(dataset)
print(f"\n✓ Total examples: {len(dataset)}")

# Split into train and validation
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset['train']
eval_dataset = dataset['test']

print(f"✓ Training examples: {len(train_dataset)}")
print(f"✓ Validation examples: {len(eval_dataset)}")


Dataset structure:
Dataset({
    features: ['text'],
    num_rows: 45
})

✓ Total examples: 45
✓ Training examples: 40
✓ Validation examples: 5


In [13]:
print("\n" + "=" * 60)
print("Sample Training Example:")
print("=" * 60)
print(train_dataset[0]['text'][:500] + "...")


Sample Training Example:
### Instruction:
You are an SQL expert. Given the following database schema and a natural language question, generate the correct SQL query.

### Database Schema:

CREATE TABLE cameras_table (
    camera_id SERIAL PRIMARY KEY,
    location TEXT,
    coverage_area TEXT,
    status TEXT CHECK (status IN ('Active', 'Inactive'))
);

CREATE TABLE events_table (
    event_id SERIAL PRIMARY KEY,
    timestamp TIMESTAMPTZ DEFAULT NOW(),
    camera_id INT REFERENCES cameras_table(camera_id),
    location...


In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# Prepare model for training
print("\n🔄 Preparing model for k-bit training...")
model = prepare_model_for_kbit_training(model)

# Configure LoRA
print("🔄 Configuring LoRA parameters...")
lora_config = LoraConfig(
    r=16,                          # Rank of the update matrices
    lora_alpha=32,                 # Scaling factor
    target_modules=[               # Modules to apply LoRA to
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,             # Dropout probability
    bias="none",                   # Bias training strategy
    task_type="CAUSAL_LM"          # Task type
)

# Apply LoRA to model
print("🔄 Applying LoRA to model...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n✓ Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"✓ Total parameters: {total_params:,}")



🔄 Preparing model for k-bit training...
🔄 Configuring LoRA parameters...
🔄 Applying LoRA to model...

✓ Trainable parameters: 39,976,960 (1.13%)
✓ Total parameters: 3,542,487,040


In [15]:
print("\n🔄 Preparing tokenization...")
def tokenize_function(examples):
    # Tokenize the text
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
    # Set labels for language modeling (copy of input_ids)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize datasets
print("🔄 Tokenizing training dataset...")
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train dataset"
)

print("🔄 Tokenizing validation dataset...")
tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_dataset.column_names,
    desc="Tokenizing eval dataset"
)

print(f"✓ Tokenized {len(tokenized_train)} training examples")
print(f"✓ Tokenized {len(tokenized_eval)} validation examples")

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)


🔄 Preparing tokenization...
🔄 Tokenizing training dataset...


Tokenizing train dataset:   0%|          | 0/40 [00:00<?, ? examples/s]

🔄 Tokenizing validation dataset...


Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

✓ Tokenized 40 training examples
✓ Tokenized 5 validation examples


In [17]:
print("=" * 60)
print("STEP 3: Configuring Fine-tuning with LoRA")
print("=" * 60)

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# Prepare model for training
print("\n🔄 Preparing model for k-bit training...")
model = prepare_model_for_kbit_training(model)

# Configure LoRA
print("🔄 Configuring LoRA parameters...")
lora_config = LoraConfig(
    r=16,                          # Rank of the update matrices
    lora_alpha=32,                 # Scaling factor
    target_modules=[               # Modules to apply LoRA to
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,             # Dropout probability
    bias="none",                   # Bias training strategy
    task_type="CAUSAL_LM"          # Task type
)

# Apply LoRA to model
print("🔄 Applying LoRA to model...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n✓ Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
print(f"✓ Total parameters: {total_params:,}")

# Tokenization function
print("\n🔄 Preparing tokenization...")
def tokenize_function(examples):
    # Tokenize the text
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
    # Set labels for language modeling (copy of input_ids)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize datasets
print("🔄 Tokenizing training dataset...")
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train dataset"
)

print("🔄 Tokenizing validation dataset...")
tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=eval_dataset.column_names,
    desc="Tokenizing eval dataset"
)

print(f"✓ Tokenized {len(tokenized_train)} training examples")
print(f"✓ Tokenized {len(tokenized_eval)} validation examples")

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're doing causal LM, not masked LM
)

# Training arguments
print("\n🔄 Configuring training arguments...")
output_dir = "./deepseek-sql-finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,                    # Number of epochs
    per_device_train_batch_size=1,         # Batch size per device
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,         # Accumulate gradients over 4 steps
    learning_rate=2e-4,                    # Learning rate
    fp16=True,                             # Use mixed precision
    save_strategy="epoch",                 # Save checkpoint every epoch
    eval_strategy="epoch",                 # FIXED: Changed from evaluation_strategy
    logging_steps=10,                      # Log every 10 steps
    warmup_steps=10,                       # Warmup steps
    save_total_limit=2,                    # Keep only 2 checkpoints
    load_best_model_at_end=True,          # Load best model at end
    metric_for_best_model="eval_loss",     # Metric to determine best model
    greater_is_better=False,               # Lower loss is better
    report_to="none",                      # Don't report to wandb/tensorboard
    optim="paged_adamw_8bit",             # Use 8-bit optimizer
)

print("✓ Training configuration:")
print(f"  - Epochs: {training_args.num_train_epochs}")
print(f"  - Batch size: {training_args.per_device_train_batch_size}")
print(f"  - Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  - Learning rate: {training_args.learning_rate}")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

# Initialize Trainer
print("\n🔄 Initializing trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

print("✓ Trainer initialized successfully!")

# Start training
print("\n" + "=" * 60)
print("STARTING TRAINING")
print("=" * 60)

# Train the model
trainer.train()

print("\n" + "=" * 60)
print("TRAINING COMPLETE ✓")
print("=" * 60)

# Save the fine-tuned model
print("\n🔄 Saving fine-tuned model...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to: {output_dir}")

# Show training results
print("\n" + "=" * 60)
print("Training Results:")
print("=" * 60)
metrics = trainer.state.log_history
if metrics:
    final_train_loss = [m for m in metrics if 'loss' in m][-1].get('loss', 'N/A')
    final_eval_loss = [m for m in metrics if 'eval_loss' in m][-1].get('eval_loss', 'N/A')
    print(f"Final Training Loss: {final_train_loss}")
    print(f"Final Validation Loss: {final_eval_loss}")

print("\n" + "=" * 60)
print("STEP 3 COMPLETE ✓")
print("=" * 60)


STEP 3: Configuring Fine-tuning with LoRA

🔄 Preparing model for k-bit training...
🔄 Configuring LoRA parameters...
🔄 Applying LoRA to model...

✓ Trainable parameters: 39,976,960 (1.13%)
✓ Total parameters: 3,542,487,040

🔄 Preparing tokenization...
🔄 Tokenizing training dataset...


Tokenizing train dataset:   0%|          | 0/40 [00:00<?, ? examples/s]

🔄 Tokenizing validation dataset...


Tokenizing eval dataset:   0%|          | 0/5 [00:00<?, ? examples/s]

✓ Tokenized 40 training examples
✓ Tokenized 5 validation examples

🔄 Configuring training arguments...
✓ Training configuration:
  - Epochs: 3
  - Batch size: 1
  - Gradient accumulation: 4
  - Learning rate: 0.0002
  - Effective batch size: 4

🔄 Initializing trainer...
✓ Trainer initialized successfully!

STARTING TRAINING


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,0.661500,0.328545
2,0.126100,0.046619
3,0.042500,0.042854



TRAINING COMPLETE ✓

🔄 Saving fine-tuned model...
✓ Model saved to: ./deepseek-sql-finetuned

Training Results:
Final Training Loss: 0.0425
Final Validation Loss: 0.04285449534654617

STEP 3 COMPLETE ✓


In [18]:
# STEP 4: Testing the Fine-tuned Model


print("=" * 60)
print("STEP 4: Testing Fine-tuned Model")
print("=" * 60)

import torch

# Set model to evaluation mode
model.eval()

# Schema definition for testing
schema_definition = """
CREATE TABLE cameras_table (
    camera_id SERIAL PRIMARY KEY,
    location TEXT,
    coverage_area TEXT,
    status TEXT CHECK (status IN ('Active', 'Inactive'))
);

CREATE TABLE events_table (
    event_id SERIAL PRIMARY KEY,
    timestamp TIMESTAMPTZ DEFAULT NOW(),
    camera_id INT REFERENCES cameras_table(camera_id),
    location TEXT,
    event_type TEXT CHECK (event_type IN ('fall', 'fight', 'entry', 'crowd', 'violation')),
    anomaly_score FLOAT CHECK (anomaly_score BETWEEN 0 AND 1),
    description TEXT,
    processed BOOLEAN DEFAULT FALSE
);

CREATE TABLE detected_objects_table (
    object_id SERIAL PRIMARY KEY,
    event_id INT REFERENCES events_table(event_id) ON DELETE CASCADE,
    object_type TEXT CHECK (object_type IN ('person', 'vehicle', 'object')),
    confidence_score FLOAT CHECK (confidence_score BETWEEN 0 AND 1),
    bounding_box JSONB
);

CREATE TABLE images_table (
    image_id SERIAL PRIMARY KEY,
    event_id INT REFERENCES events_table(event_id) ON DELETE CASCADE,
    file_path TEXT,
    thumbnail_path TEXT,
    captured_at TIMESTAMPTZ DEFAULT NOW()
);
"""

def generate_sql(question, schema=schema_definition):
    """Generate SQL query from natural language question"""

    prompt = f"""### Instruction:
You are an SQL expert. Given the following database schema and a natural language question, generate the correct SQL query.

### Database Schema:
{schema}

### Question:
{question}

### SQL Query:"""

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate output
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.1,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode output
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract SQL query (everything after "### SQL Query:")
    if "### SQL Query:" in full_output:
        sql_query = full_output.split("### SQL Query:")[-1].strip()
        # Clean up any extra text after the query
        if "\n\n" in sql_query:
            sql_query = sql_query.split("\n\n")[0].strip()
        return sql_query
    else:
        return full_output.strip()

# Test queries
test_questions = [
    "Show me all events",
    "Get all active cameras",
    "Show all fall events",
    "Find events with high anomaly score above 0.8",
    "Show events from today",
    "Count events by type",
    "Show events with camera details",
    "Find fight or fall events with high anomaly score",
    "Show latest events first",
    "Get top 5 high severity events",
    "Show unprocessed events from last hour",
    "Find events where students are fighting",
]

print("\n" + "=" * 60)
print("Testing Model with Various Questions")
print("=" * 60)

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. Question: {question}")
    print("-" * 60)
    sql = generate_sql(question)
    print(f"Generated SQL:\n{sql}")
    print()

print("=" * 60)
print("STEP 4 COMPLETE ✓")
print("=" * 60)

# Interactive testing function
print("\n" + "=" * 60)
print("Interactive Testing Function Ready!")
print("=" * 60)


STEP 4: Testing Fine-tuned Model

Testing Model with Various Questions

1. Question: Show me all events
------------------------------------------------------------
Generated SQL:
SELECT * FROM events_table;


2. Question: Get all active cameras
------------------------------------------------------------
Generated SQL:
SELECT * FROM cameras_table WHERE status = 'Active';


3. Question: Show all fall events
------------------------------------------------------------
Generated SQL:
SELECT * FROM events_table WHERE event_type = 'fall';


4. Question: Find events with high anomaly score above 0.8
------------------------------------------------------------
Generated SQL:
SELECT * FROM events_table WHERE anomaly_score > 0.8;


5. Question: Show events from today
------------------------------------------------------------
Generated SQL:
SELECT * FROM events_table WHERE DATE(timestamp) = CURRENT_DATE;


6. Question: Count events by type
-----------------------------------------------------

In [20]:
# STEP 5: Model Accuracy and Precision Evaluation
# Run this cell after Step 4 completes successfully

print("=" * 60)
print("=" * 60)

import torch
from difflib import SequenceMatcher
import re

def clean_sql(sql):
    """Clean and normalize SQL for comparison"""
    # Remove extra whitespace
    sql = re.sub(r'\s+', ' ', sql.strip())
    # Convert to lowercase for comparison
    sql = sql.lower()
    # Remove semicolons
    sql = sql.replace(';', '')
    return sql

def calculate_sql_similarity(generated, expected):
    """Calculate similarity between generated and expected SQL"""
    gen_clean = clean_sql(generated)
    exp_clean = clean_sql(expected)

    # Use SequenceMatcher for similarity
    similarity = SequenceMatcher(None, gen_clean, exp_clean).ratio()
    return similarity

def is_sql_correct(generated, expected, threshold=0.85):
    """Check if generated SQL is correct (above threshold)"""
    similarity = calculate_sql_similarity(generated, expected)
    return similarity >= threshold, similarity

# Comprehensive test set with ground truth
test_cases = [
    # Basic SELECT queries
    {
        "question": "Show me all events",
        "expected": "SELECT * FROM events_table;"
    },
    {
        "question": "Get all active cameras",
        "expected": "SELECT * FROM cameras_table WHERE status = 'Active';"
    },
    {
        "question": "List all camera locations",
        "expected": "SELECT camera_id, location FROM cameras_table;"
    },

    # Event type queries
    {
        "question": "Show all fall events",
        "expected": "SELECT * FROM events_table WHERE event_type = 'fall';"
    },
    {
        "question": "Find all fight incidents",
        "expected": "SELECT * FROM events_table WHERE event_type = 'fight';"
    },
    {
        "question": "Get crowd detection events",
        "expected": "SELECT * FROM events_table WHERE event_type = 'crowd';"
    },

    # Anomaly score queries
    {
        "question": "Show events with high anomaly score above 0.8",
        "expected": "SELECT * FROM events_table WHERE anomaly_score > 0.8;"
    },
    {
        "question": "Find events with anomaly score greater than 0.9",
        "expected": "SELECT * FROM events_table WHERE anomaly_score > 0.9;"
    },

    # Time-based queries
    {
        "question": "Show events from today",
        "expected": "SELECT * FROM events_table WHERE DATE(timestamp) = CURRENT_DATE;"
    },
    {
        "question": "Get events from last 24 hours",
        "expected": "SELECT * FROM events_table WHERE timestamp >= NOW() - INTERVAL '24 hours';"
    },

    # JOIN queries
    {
        "question": "Show events with camera details",
        "expected": "SELECT e.*, c.location, c.coverage_area FROM events_table e JOIN cameras_table c ON e.camera_id = c.camera_id;"
    },
    {
        "question": "Get events with detected objects",
        "expected": "SELECT e.event_id, e.event_type, d.object_type, d.confidence_score FROM events_table e JOIN detected_objects_table d ON e.event_id = d.event_id;"
    },

    # Aggregation queries
    {
        "question": "Count total events",
        "expected": "SELECT COUNT(*) FROM events_table;"
    },
    {
        "question": "Count events by type",
        "expected": "SELECT event_type, COUNT(*) as count FROM events_table GROUP BY event_type;"
    },
    {
        "question": "Get average anomaly score",
        "expected": "SELECT AVG(anomaly_score) as avg_score FROM events_table;"
    },

    # Complex queries
    {
        "question": "Show unprocessed high severity events",
        "expected": "SELECT * FROM events_table WHERE processed = FALSE AND anomaly_score > 0.8;"
    },
    {
        "question": "Find fight or fall events with high anomaly score",
        "expected": "SELECT * FROM events_table WHERE event_type IN ('fight', 'fall') AND anomaly_score > 0.8;"
    },
    {
        "question": "Get person detections with high confidence",
        "expected": "SELECT * FROM detected_objects_table WHERE object_type = 'person' AND confidence_score > 0.85;"
    },

    # ORDER BY queries
    {
        "question": "Show latest events first",
        "expected": "SELECT * FROM events_table ORDER BY timestamp DESC;"
    },
    {
        "question": "List events by highest anomaly score",
        "expected": "SELECT * FROM events_table ORDER BY anomaly_score DESC;"
    },

    # LIMIT queries
    {
        "question": "Show last 5 events",
        "expected": "SELECT * FROM events_table ORDER BY timestamp DESC LIMIT 5;"
    },
    {
        "question": "Get top 5 high severity events",
        "expected": "SELECT * FROM events_table ORDER BY anomaly_score DESC LIMIT 5;"
    },
]

print(f"\n🔄 Testing model on {len(test_cases)} queries...")
print("=" * 60)

# Evaluate model
results = []
correct_count = 0
total_similarity = 0

for i, test_case in enumerate(test_cases, 1):
    question = test_case["question"]
    expected = test_case["expected"]

    # Generate SQL
    generated = generate_sql(question)

    # Calculate correctness
    is_correct, similarity = is_sql_correct(generated, expected)

    results.append({
        "question": question,
        "expected": expected,
        "generated": generated,
        "similarity": similarity,
        "correct": is_correct
    })

    if is_correct:
        correct_count += 1
    total_similarity += similarity

    # Print result
    status = "✓" if is_correct else "✗"
    print(f"{status} Test {i}/{len(test_cases)} - Similarity: {similarity:.2%}")
    if not is_correct:
        print(f"   Question: {question}")
        print(f"   Expected: {expected[:80]}...")
        print(f"   Got: {generated[:80]}...")
    print()

# Calculate metrics
accuracy = correct_count / len(test_cases)
avg_similarity = total_similarity / len(test_cases)

# Calculate precision (for queries that were marked correct)
true_positives = correct_count
false_positives = len(test_cases) - correct_count
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

print("\n" + "=" * 60)
print("EVALUATION RESULTS")
print("=" * 60)
print(f"\nTotal Test Cases: {len(test_cases)}")
print(f"Correct Predictions: {correct_count}")
print(f"Incorrect Predictions: {len(test_cases) - correct_count}")
print(f"\n{'='*60}")
print(f"ACCURACY: {accuracy:.2%}")
print(f"PRECISION: {precision:.2%}")
print(f"AVERAGE SIMILARITY: {avg_similarity:.2%}")
print(f"{'='*60}")

# Breakdown by query type
print("\n" + "=" * 60)
print("Performance Breakdown:")
print("=" * 60)

query_types = {
    "Basic SELECT": ["Show me all events", "Get all active cameras", "List all camera locations"],
    "Event Type Filters": ["Show all fall events", "Find all fight incidents", "Get crowd detection events"],
    "Anomaly Score": ["Show events with high anomaly score", "Find events with anomaly score"],
    "Time-based": ["Show events from today", "Get events from last 24 hours"],
    "JOINs": ["Show events with camera details", "Get events with detected objects"],
    "Aggregations": ["Count total events", "Count events by type", "Get average anomaly score"],
    "Complex": ["Show unprocessed high severity events", "Find fight or fall events", "Get person detections"],
    "Sorting": ["Show latest events first", "List events by highest anomaly score"],
    "LIMIT": ["Show last 10 events", "Get top 5 high severity events"]
}

for query_type, keywords in query_types.items():
    type_results = [r for r in results if any(kw in r["question"] for kw in keywords)]
    if type_results:
        type_accuracy = sum(1 for r in type_results if r["correct"]) / len(type_results)
        type_similarity = sum(r["similarity"] for r in type_results) / len(type_results)
        print(f"\n{query_type}:")
        print(f"  Tests: {len(type_results)}")
        print(f"  Accuracy: {type_accuracy:.2%}")
        print(f"  Avg Similarity: {type_similarity:.2%}")

# Show examples of correct and incorrect predictions
print("\n" + "=" * 60)
print("Sample Correct Predictions:")
print("=" * 60)
correct_samples = [r for r in results if r["correct"]][:3]
for sample in correct_samples:
    print(f"\nQuestion: {sample['question']}")
    print(f"Generated: {sample['generated']}")
    print(f"Similarity: {sample['similarity']:.2%}")

print("\n" + "=" * 60)
print("Sample Incorrect Predictions (if any):")
print("=" * 60)
incorrect_samples = [r for r in results if not r["correct"]][:3]
if incorrect_samples:
    for sample in incorrect_samples:
        print(f"\nQuestion: {sample['question']}")
        print(f"Expected: {sample['expected']}")
        print(f"Generated: {sample['generated']}")
        print(f"Similarity: {sample['similarity']:.2%}")
else:
    print("\n✓ No incorrect predictions! Perfect score!")

print("\n" + "=" * 60)
print("EVALUATION COMPLETE ✓")
print("=" * 60)

# Save results summary
summary = f"""
MODEL EVALUATION SUMMARY
========================
Training Data: 47 examples (42 train, 5 validation)
Model: DeepSeek Coder 6.7B (Fine-tuned with LoRA)
Training Epochs: 3
Final Training Loss: 0.0425
Final Validation Loss: 0.0428

Test Performance:
- Total Test Cases: {len(test_cases)}
- Correct Predictions: {correct_count}
- ACCURACY: {accuracy:.2%}
- PRECISION: {precision:.2%}
- AVERAGE SIMILARITY: {avg_similarity:.2%}

The model successfully converts natural language queries to SQL
with high accuracy for surveillance system database operations.
"""

print("\n" + summary)

# Store for later use
evaluation_results = {
    "accuracy": accuracy,
    "precision": precision,
    "avg_similarity": avg_similarity,
    "total_tests": len(test_cases),
    "correct": correct_count,
    "summary": summary
}


🔄 Testing model on 22 queries...
✓ Test 1/22 - Similarity: 100.00%

✓ Test 2/22 - Similarity: 100.00%

✓ Test 3/22 - Similarity: 86.08%

✓ Test 4/22 - Similarity: 100.00%

✓ Test 5/22 - Similarity: 100.00%

✓ Test 6/22 - Similarity: 100.00%

✓ Test 7/22 - Similarity: 100.00%

✓ Test 8/22 - Similarity: 100.00%

✓ Test 9/22 - Similarity: 100.00%

✓ Test 10/22 - Similarity: 99.31%

✗ Test 11/22 - Similarity: 46.02%
   Question: Show events with camera details
   Expected: SELECT e.*, c.location, c.coverage_area FROM events_table e JOIN cameras_table c...
   Got: SELECT events_table.event_id, events_table.timestamp, events_table.location, eve...

✗ Test 12/22 - Similarity: 45.49%
   Question: Get events with detected objects
   Expected: SELECT e.event_id, e.event_type, d.object_type, d.confidence_score FROM events_t...
   Got: SELECT * FROM events_table WHERE event_id IN (SELECT event_id FROM detected_obje...

✓ Test 13/22 - Similarity: 100.00%

✓ Test 14/22 - Similarity: 93.53%

✓ Test 